In [4]:
import torch


def reward(
    task_score: float,
    language_score: float,
    t_weight: float = 1.0,
    l_weight: float = 1.0,
) -> float:
    """
    Computes the harmonic weighted reward score based on task success and language aspect.

    This method calculates a reward by combining the task performance score 
    with a penalty based on the language aspect. The final score is normalized 
    to the range [0, 1].

    Parameters
    ----------
    task_score : float
        A score in the range [0, 1] indicating task performance.
    language_score : float
        A score in the range [0, 1] indicating adherence to language constraints. It equals in our cases often to 1 - penalisation_score, where penalisation_score is included between 0 and 1.
    t_weight : float, optional
        The weight assigned to the task score (default is 1.0).
    l_weight : float, optional
        The weight assigned to the language score penalty (default is 1.0).

    Returns
    -------
    float
        A normalized reward score in the range [0, 1].

    Notes
    -----
    - First implementation was using a weighted average score, which induced the following issue : a random agent would have a constant reward of 0.5, whereas , in pricing game, a failing calculation of bid additioned to using P1 information was leading to a reward of 0. 
    In math task example, saying random things without using equal sign was leading as well to a reward 0.5, where trying to solve the problem was inducing a worst reward. We hope to fix this problematic case with a weighted harmonic reward.
    - A higher `task_score` increases the reward.
    - A higher `language_score` decreases the reward.
    - A low task score or language score induces a low reward.
    - Custom reward functions can be implemented by overriding this method.
    """
    r = (t_weight + l_weight) * task_score * language_score / (t_weight * language_score + l_weight * task_score + torch.finfo(torch.float32).eps)

    return r

In [12]:
reward(0.3, 1)

0.4615384192156407

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [ ]:
token_ids = tokenizer("""I need to determine the total cost of the product, including all factors provided.
The procurement department negotiated a base material cost of $23.735 per unit. Additionally, overhead costs add 31.445% to production costs. There's also a volume incentive of 2.742% per unit. 

I'll start by calculating the overhead cost. I'll take the base material cost and multiply it by 31.445% to find the additional overhead. Then, I'll add this amount to the base material cost to get the total production cost.

Next, I'll include the volume incentive. I'll take the total production cost and add 2.742% of that cost to account for the volume incentive.

Finally, I'll add the profit margin of 28% to the adjusted price to determine the final price I will bid. This final price will be the amount I will offer for the product.""", add_special_tokens=False)["input_ids"]

In [18]:
len(token_ids)

181